<center><font size="24" face="Helvetica"><b>A toy model for <tt>ALD-SDDiP</tt></b></font></center>

</br>

![CC-BY-SA](https://mirrors.creativecommons.org/presskit/buttons/88x31/svg/by-sa.svg)
This notebook was created by [Bernardo Freitas Paulo da Costa](http://www.im.ufrj.br/bernardofpc),
and is licensed under Creative Commons BY-SA

# Libs

In [ ]:
import SDDP, JuMP, PyPlot
import SLDP

In [ ]:
import LaTeXStrings: @L_str

## Local utils

In [ ]:
include("control.jl")
include("plt_control.jl");

# Problem description

We consider a very simple stochastic control problem, with
- 1-dimensional state $x_t$, subject to
    - "Binary" control $c_t = \pm 1$; and
    - uncertain displacement $\xi_t$.
- Cost proportional to the distance to zero, subject to
    - depreciation decay factor $\beta$.

## Formulation

$$\begin{array}{rl}
      \min  & \mathbb{E}\left[\sum\limits_t \beta^{t-1}|x_t|\right] \\
\text{s.t.} & \quad x_t = x_{t-1} + c_t + \xi_t \\
            & \quad c_t \in \{\pm 1\}
\end{array} $$

## Future cost functions

The corresponding dynamic programming / recursive equations are:

$$ Q_t(x_{t-1},\xi_t) =
  \begin{array}[t]{rl}
  \min\limits_{x_t} & |x_t| + \beta \cdot \mathbb{E}\left[ Q_{t+1}(x_t,\xi_{t+1}) \right] \\
  \text{s.t.}       & x_t = x_{t-1} + c_t + \xi_t \\
                    & c_t \in \{\pm1\}
  \end{array}
$$

The averages will be denoted (as usual) by $\overline{Q}_t(x_{t-1}) = \mathbb{E}\big[ Q_t(x_{t-1},\xi_t) \big]$.

### Cuts and lower approximations

- Any lower bound for $\overline{Q}_t$ is a _cut_.
- The maximum of the $k$ first cuts is denoted $\mathfrak{Q}_t^k$,
  and is constructed incrementally during the SDDP algorithm.
- The dynamic programming where we replace $\overline{Q}_{t+1}$ with $\mathfrak{Q}_{t+1}$
  yields the _backwards functions_:
$$ \tilde{Q}_t^k(x_{t-1},\xi_t) =
  \begin{array}[t]{rl}
  \min\limits_{x_t} & |x_t| + \beta \cdot \mathfrak{Q}_{t+1}^k(x_t) \\
  \text{s.t.}       & x_t = x_{t-1} + c_t + \xi_t \\
                    & c_t \in \{\pm1\}
  \end{array}
$$

## Data 

### The discount factor $\beta$

We take a relatively "small" decay, $\beta = 0.9$.

In [ ]:
discount = 0.9;

### The noise $\xi_t$

Is identically distributed ("periodic system"), and **symmetric** (important for the analytic solution).

In [ ]:
srand(11111)
A_noise   = 0.4
num_noise = 5
noise = randn(num_noise)
noise = A_noise * [noise; -noise];

# Analytic solution by symmetry

## Calculations for a 61-point discretization

In [ ]:
ts = -3:0.1:3;

In [ ]:
Qt3 = Vector{Float64}[]
for t = 1:7
    v = [Q2_bar(ti, 8, noise, discount, t) for ti in ts]
    push!(Qt3, v)
end

In [ ]:
fig, (ax1,ax2) = PyPlot.subplots(ncols=2, figsize=(10,4))
for t = 1:7
    ax1[:plot](ts, Qt3[t], label="$t")
    ax2[:plot](ts, Qt3[t]*discount^t, label="$t")
end
ax1[:set_title]("Future cost (value at current time)")
ax2[:set_title]("Future cost (in 1st stage values)")
ax2[:legend](title=L"Stage $t$", bbox_to_anchor=(1,0.5), loc="center left");

The future cost "at current time" can also be interpreted as the future cost at the first stage considering a time horizon of $9-t$ stages.
So, for example, the line corresponding to "stage 3" is also the future cost function at the first stage of a 6-stage problem.

The limit of these curves can be taken as one approximation for the "infinite horizon problem".

### Derivatives and Lip constant

In [ ]:
fig, (ax1,ax2) = PyPlot.subplots(ncols=2, figsize=(10,4))
for t = 1:7
    tmp = Qt3[t]
    deriv = (tmp[3:end]-tmp[1:end-2])/0.2
    ax1[:plot](ts[2:end-1], deriv, label="$t")
    ax2[:plot](ts[2:end-1], deriv*discount^t, label="$t")
end
ax1[:set_title]("Derivative of future cost (value at current time)")
ax2[:set_title]("Derivative of future cost (in 1st stage values)")
ax2[:legend](title=L"Stage $t$", bbox_to_anchor=(1,0.5), loc="center left");

In [ ]:
fig, (ax1,ax2) = PyPlot.subplots(ncols=2, figsize=(10,4))
for t = 1:7
    tmp = Qt3[t]
    zoomts = ts[21:41]
    zoomderiv = (tmp[22:42]-tmp[20:40])/0.2
    ax1[:plot](zoomts, zoomderiv, label="$t")
    ax2[:plot](zoomts, zoomderiv*discount^t, label="$t")
end
ax1[:set_title]("Derivative of future cost (value at current time)")
ax2[:set_title]("Derivative of future cost (in 1st stage values)")
ax2[:legend](title=L"Stage $t$", bbox_to_anchor=(1,0.5), loc="center left");

### Cost-to-go and Average cost-to-go

In [ ]:
fig, (ax1,ax2) = PyPlot.subplots(ncols=2, figsize=(10,4), sharey=true)
for t = 1:7
    ctg = [solve_bin_sym(ti, 8, noise, discount, t) for ti in ts]
    ax1[:plot](ts, ctg*discount^(t-1), label="$t")
    ax2[:plot](ts, Qt3[t]*discount^t, label="$t")
end
ax1[:set_title]("Cost-to-go")
ax2[:set_title](L"Future cost $\overline{Q}_t(x_{t-1})$")
ax1[:set_xlabel](L"$x_{t-1} + \xi_t$")
ax2[:set_xlabel](L"$x_{t-1}$")
ax2[:legend](title=L"Stage $t$", bbox_to_anchor=(1,0.5), loc="center left");

# Load models

In [ ]:
names      = ["SB", "ALD simple", "ALD parallel", "ALD parallel2"]
ramp_modes = [:None, :simple, :parallel, :parallel2]
models     = [controlmodel(nstages=8, discount=discount, ramp_mode=r, noise=noise) for r in ramp_modes];

In [ ]:
for (model,name) in zip(models,names)
    base = joinpath("cuts", name)
    SDDP.loadcuts!(model, base*"_benders.cuts")
    SLDP.load_aldcuts!(model, base*"_ald.cuts")
    
    SLDP.readlog!(model, joinpath("logs", name)*".log")
end

# Results per cut strategy

## Baseline strategy: only Strenghtened Benders cuts

This corresponds to $\rho = 0$, for all stages and iterations.

In [ ]:
sb_model = models[1];

In [ ]:
ts = -3:0.02:3
for t = 1:7
    PyPlot.plot(ts, SLDP.Qfrak(sb_model,t,1,ts), label="$t")
end
PyPlot.legend(title=L"Stage $t$")
PyPlot.title(L"Future cost functions $\mathfrak{Q}_t$")
PyPlot.grid();

In [ ]:
ts = -3:0.1:3
for t = 1:7
    QTrue = Qt3[t]*discount^t
    graph_fcfs(sb_model,t, ts, QTrue, filename=joinpath("figs", names[1]*"_stage$t.pdf"))
end

## Lipschitz estimate

We estimate $\displaystyle Lip_t = \beta^{t-1} + \beta^t + \ldots + \beta^{8-1} = \beta^{t-1}(1 + \ldots + \beta^{8-t}) = \frac{1 - \beta^{8+1-t}}{1 - \beta}\beta^{t-1}$.

## First strategy: homothetic ALD

At iteration $n$ and stage $t$, we set
$$\rho_t = \min\left(1, \max\left(0, \frac{n-15}{15}\right)\right) \cdot Lip_t. $$

That is:

- in the first 15 stages, $\rho_t = 0$;
- then increase $\rho_t$ until it reaches $Lip_t$ in 15 stages;
- and keep at $Lip_t$ until the end.

In [ ]:
ald_simple_model = models[2];

In [ ]:
ts = -3:0.02:3
for t = 1:7
    PyPlot.plot(ts, SLDP.Qfrak(ald_simple_model,t,1,ts), label="$t")
end
PyPlot.legend(title="Stage")
PyPlot.title("Future cost functions")
PyPlot.grid();

In [ ]:
ts = -3:0.1:3
for t = 1:7
    QTrue = Qt3[t]*discount^t
    graph_fcfs(ald_simple_model,t, ts, QTrue, filename=joinpath("figs",names[2]*"_stage$t.pdf"))
end

### Comparing 100 and 200 iterations

That is, (16 SB + 84 ALD) _vs_ (16 SB + 184 ALD).

Remember that for every ALD cut we also add the corresponding SB cut,
so in effect there are $16 + 2\cdot84 = 184$ and $16 + 2\cdot184 = 384$ cuts.

In [ ]:
ts = -3:0.02:3
us = -3:0.1:3
fig, axs = PyPlot.subplots(ncols=2,nrows=4, sharey=true, figsize=(12,14))
for t = 1:7
    curax = axs[div(t+1,2),rem(t-1,2)+1]
    f1 = SLDP.Qfrak(models[2],t,1,ts)
    f2 = SLDP.Qfrak(models[2],t,1,ts, ncuts=100,nald=84)
    curax[:plot](ts, f1, label="all cuts")
    curax[:plot](ts, f2, label="16 SB + 84 ALD cuts")
    curax[:plot](us, Qt3[t]*discount^t, label="True")
    curax[:legend]()
    curax[:set_title]("Functions at stage $t")
    curax[:grid]()
end

In [ ]:
ts = -3:0.02:3
fig, axs = PyPlot.subplots(ncols=2,nrows=7, sharey=true, figsize=(12,23))
fig[:suptitle]("Difference between 100 and 200 iterations")
for t = 1:7
    ax1 = axs[t,1]
    ax2 = axs[t,2]
    f1 = SLDP.Qfrak(models[2],t,1,ts)
    f2 = SLDP.Qfrak(models[2],t,1,ts, ncuts=100,nald=84)
    ax1[:plot](ts, f1 - f2)
    ax1[:set_title]("Stage $t, in present values")
    ax1[:grid]()
    ax2[:plot](ts, (f1 - f2)/discount^(t-1))
    ax2[:set_title]("Stage $t, in current values")
    ax2[:grid]()
end

## Second strategy: parallel

At iteration $n$ and stage $t$, we set
$$\rho_t = \min\left(Lip_t, \max\left(0, \frac{n-15}{15}\right)\right). $$

That is:

- in the first 15 stages, $\rho_t = 0$;
- then increase with equal steps at all stages, stopping at $Lip_t$; (so that different stages "saturate" at different times)
- and keep at $Lip_t$ until the end.

In [ ]:
ald_par_model = models[3];

In [ ]:
ts = -3:0.02:3
for t = 1:7
    PyPlot.plot(ts, SLDP.Qfrak(ald_par_model,t,1,ts), label="$t")
end
PyPlot.legend(title="Stage")
PyPlot.title("Future cost functions")
PyPlot.grid();

In [ ]:
ts = -3:0.1:3
for t = 1:7
    QTrue = Qt3[t]*discount^t
    graph_fcfs(ald_par_model,t, ts, QTrue, filename=joinpath("figs", names[3]*"_stage$t.pdf"))
end

### Creating cuts

We can observe that $\tilde{Q}_6$ is still a close approximation to $\overline{Q}_6$,
but $\mathfrak{Q}_6$ is a very poor approximation outside of $[-1,1]$.
One could conjecture that the "gap" that (progressively) opens between $\overline{Q}_t$ and $\tilde{Q}_t$ in previous stages
comes from these gaps outside of $[-1,1]$.

Let's see if closing these gaps manually yields better approximations.

In [ ]:
t = 7
stage6 = SDDP.getstage(ald_par_model,t-1)
stage6.state
stage7 = SDDP.getstage(ald_par_model,t)
rho7 = stage7.ext[:rhos](100, stage7.ext[:Lip])

In [ ]:
SLDP.make_cut(ald_par_model,7,[-2.], rho7)

In [ ]:
SLDP.make_cut(ald_par_model,7,[ 2.], rho7)

In [ ]:
for x in [-1.5,1.5]
    SLDP.make_cut(ald_par_model,7,[x],0.1)
end

In [ ]:
for x in [-1.25,1.25]
    SLDP.make_cut(ald_par_model,7,[x],0.4)
end

In [ ]:
graph_fcfs(ald_par_model, 6, ts, Qt3[6]*discount^6)

#### Impact on previous stage cost function

In [ ]:
ts = -3:0.1:3
graph_fcfs(ald_par_model, 5, ts, Qt3[5]*discount^5)

## Third strategy: double of parallel

At iteration $n$ and stage $t$, we set
$$\rho_t = \min\left(2 Lip_t, \max\left(0, \frac{n-15}{15}\right)\right). $$

That is:

- in the first 15 stages, $\rho_t = 0$;
- then increase with equal steps at all stages, stopping at $2 Lip_t$; (so that different stages "saturate" at different times)
- and keep at $2 Lip_t$ until the end.

In [ ]:
ald_par2_model = models[4];

In [ ]:
ts = -3:0.02:3
for t = 1:7
    PyPlot.plot(ts, SLDP.Qfrak(ald_par2_model,t,1,ts), label="$t")
end
PyPlot.legend(title="Stage")
PyPlot.title("Future cost functions")
PyPlot.grid();

In [ ]:
ts = -3:0.1:3
for t = 1:7
    QTrue = Qt3[t]*discount^t
    graph_fcfs(ald_par2_model,t, ts, QTrue, filename=joinpath("figs", names[4]*"_stage$t.pdf"))
end

# Comparing models

## $\mathfrak{Q}_1$

In [ ]:
t = 1
compare_qfrak(models, t, ts, Qt3[t]*discount^t)

## $\mathfrak{Q}_3$

In [ ]:
t = 3
compare_qfrak(models, t, ts, Qt3[t]*discount^t)

## Lower bound evolution

In [ ]:
fig, (ax1,ax2) = PyPlot.subplots(ncols=2, figsize=(12,4))
PyPlot.suptitle("Lower bound")

for (i,m) in enumerate(models)
    mylog = m.log
    ts = (x -> x.timecuts).(mylog)
    vs = (x -> x.bound).(mylog)
    ax1[:plot](ts - mylog[1].timecuts, vs, label="$i")
    ax2[:semilogx](ts - mylog[1].timecuts, vs, label="$i")
end
for ax in (ax1,ax2)
    ax[:set_xlabel]("Time")
    ax[:legend](names)
end

In [ ]:
fig, (ax1,ax2) = PyPlot.subplots(ncols=2, figsize=(12,4))
PyPlot.suptitle("Lower bound")

for (i,m) in enumerate(models)
    mylog = m.log
    ts = (x -> x.timecuts).(mylog)
    vs = (x -> x.bound).(mylog)
    ax1[:plot](vs, label="$i")
    ax2[:semilogx](vs, label="$i")
end
for ax in (ax1,ax2)
    ax[:set_xlabel]("Iteration")
    ax[:legend](names)
end

## Timing

In [ ]:
fig, (ax1,ax2) = PyPlot.subplots(ncols=2, figsize=(12,4))
PyPlot.suptitle("Time per iteration")

for (i,m) in enumerate(models)
    mylog = m.log
    ts = (x -> x.timecuts).(mylog)
    vs = (x -> x.bound).(mylog)
    ax1[:plot](diff(ts), label="$i")
    ax2[:loglog](diff(ts), label="$i")
end
for ax in (ax1,ax2)
    ax[:set_xlabel]("Iteration")
    ax[:legend](names)
end

In [ ]:
for win in [5,10,20]
    fig, (ax1,ax2) = PyPlot.subplots(ncols=2, figsize=(12,4))
    PyPlot.suptitle("Time per iteration (smoothed by a window of size $win)")

    for (i,m) in enumerate(models)
        mylog = m.log
        ts = (x -> x.timecuts).(mylog)
        dts = (ts[1+win:end] - ts[1:end-win])/win
        ax1[:plot](dts, label="$i")
        ax2[:loglog](dts, label="$i")
    end
    for ax in (ax1,ax2)
        ax[:set_xlabel]("Iteration")
        ax[:legend](names)
    end
end